# Prepare CNN Data

## Setup

In [1]:
# USE ACCOUNT: robmarty3@gmail.com
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWhegSh7VhFE-GsccZelG3HH7EnmbE5i8wfhiXVRZshWfT96P8vWY-s



Successfully saved authorization token.


In [2]:
#image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
#    .filterDate("2019-01-01", "2019-02-01")\
#    .map(cloud_mask_landsatSR)\
#    .median()
#
#print(image.getInfo())

In [3]:
import numpy as np
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf
import config as cf
import ee_utils as utils
import eeconvert
import time

#import geetools
#from geetools import ui, cloud_mask
#cloud_mask_landsatSR = cloud_mask.landsatSR()
#cloud_mask_sentinel2 = cloud_mask.sentinel2()

/Users/robmarty/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/robmarty/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/Users/robmarty/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Parameters

In [4]:
#224
SURVEY_NAME = 'DHS'
SATELLITE = 'l8'
SKIP_IF_SCRAPED = True
CHUNK_SIZE = 10 # Number of observtaions to scrape in GEE at any given time
DROPBOX_DIR = cf.DROPBOX_DIRECTORY
GOOGLEDRIVE_DIR = cf.GOOGLEDRIVE_DIRECTORY

if SATELLITE == 's2':
    KERNEL_SIZE = 500
elif SATELLITE == 'l8':
    KERNEL_SIZE = 167

print(KERNEL_SIZE)

167


In [5]:
# Directory to store tfrecords
out_path = os.path.join(GOOGLEDRIVE_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE,
            'tfrecords')

## Load Data

In [6]:
# Relies on VIIRs Values
survey_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'survey_socioeconomic.csv'))

viirs_2_5km_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'survey_viirs_2_5km.csv'))
viirs_5km_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'survey_viirs_5km.csv'))

viirs_2_5km_df.viirs_avg_rad = np.log(viirs_2_5km_df.viirs_avg_rad + 1)
viirs_5km_df.viirs_avg_rad = np.log(viirs_5km_df.viirs_avg_rad + 1)

viirs_2_5km_df = viirs_2_5km_df.rename(columns={"viirs_avg_rad": "viirs_avg_rad_2_5km"})
viirs_5km_df = viirs_5km_df.rename(columns={"viirs_avg_rad": "viirs_avg_rad_5km"})

survey_df = survey_df.merge(viirs_2_5km_df, on = 'uid')
survey_df = survey_df.merge(viirs_5km_df, on = 'uid')

survey_df.shape

/Users/robmarty/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (26,30,33,36,37,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(64249, 44)

In [7]:
# Sentinel doesn't capture Lakshadweep (island off coast of mainland India)
#survey_df = survey_df[survey_df['GID_2'] != "IND.18.1_1"]

# List of TF Records
tf_record_list = list(np.unique(survey_df.tfrecord_name))

In [8]:
# Remove if issues extracting
#survey_df = survey_df[survey_df['uid'] != "IA201400180079"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180052"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180112"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180081"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180011"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180048"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180058"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180028"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180072"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180047"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180012"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180040"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180055"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180140"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180030"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180104"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180123"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180062"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180080"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180050"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180116"]

In [9]:
# If skip already scraped, remove existing tfrecords from tf_record_list
if SKIP_IF_SCRAPED:
    tf_records_exist = os.listdir(out_path)
    tf_record_list = [x for x in tf_record_list if x not in tf_records_exist]

In [ ]:
### Loop through all tfrecords
for tfr_i in tf_record_list:
    
    # Sometimes we get computational time out errors. If occurs, just skip and go to next.
    # We can then go back and rescrape missed ones.
    try:
    
        survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
        year_i = survey_df_yeari['year'].iloc[0]

        ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
        survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

        print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

        proto_examples_all = []
        for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
            time.sleep(6)
            print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

            survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

            proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
            proto_examples_all.extend(proto_examples_i)

        ### Save data as tf record
        out_path_i = os.path.join(out_path, tfr_i)
        print(out_path_i)
        with tf.io.TFRecordWriter(out_path_i) as writer:
            for tf_example in proto_examples_all:
                writer.write(tf_example.SerializeToString())
                
    except:
        pass

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Putting 203 observations into BO_5_1.tfrecord
Observation: 0/203
Observation: 10/203
Observation: 20/203
Observation: 30/203


/Users/robmarty/Documents/Github/Pakistan-Poverty-from-Sky/DataWork/02_get_process_ancillary_data/CNN Features Predict NTL/ee_utils.py:80: RuntimeWarning: divide by zero encountered in true_divide
  return (values2 - values1)/(values2 + values1)


Observation: 40/203
Observation: 50/203
Putting 63 observations into BO_5_2.tfrecord
Observation: 0/63
Observation: 10/63
Putting 200 observations into CO_1_1.tfrecord
Observation: 0/200
Observation: 10/200
Observation: 20/200
Observation: 30/200
Observation: 40/200
Observation: 50/200
Observation: 60/200
Observation: 70/200


/Users/robmarty/Documents/Github/Pakistan-Poverty-from-Sky/DataWork/02_get_process_ancillary_data/CNN Features Predict NTL/ee_utils.py:80: RuntimeWarning: invalid value encountered in true_divide
  return (values2 - values1)/(values2 + values1)


...

/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/GH_3_1.tfrecord
Putting 105 observations into GH_4_1.tfrecord
Observation: 0/105
Observation: 10/105
Observation: 20/105
Observation: 30/105
Observation: 40/105
Observation: 50/105
Observation: 60/105
Observation: 70/105
Observation: 80/105
Observation: 90/105
Observation: 100/105
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/GH_4_1.tfrecord
Putting 78 observations into GH_5_1.tfrecord
Observation: 0/78
Observation: 10/78
Observation: 20/78
Observation: 30/78
Observation: 40/78
Observation: 50/78
Observation: 60/78
Observation: 70/78
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/GH_5_1.tfrecord
Putting 52 observations into GM_1_1.tfrecord
Observation: 0/52
Observation: 10/52
Observation: 20/52
Obser

W0911 13:40:54.294146 4616400320 http.py:171] Sleeping 0.00 seconds before retry 1 of 5 for request: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json, after 502


Observation: 40/52
Observation: 50/52
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/GM_1_1.tfrecord
Putting 45 observations into GM_2_1.tfrecord
Observation: 0/45
Observation: 10/45
Observation: 20/45
Observation: 30/45
Observation: 40/45
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/GM_2_1.tfrecord
Putting 86 observations into GM_3_1.tfrecord
Observation: 0/86
Observation: 10/86


W0911 13:44:05.814131 4616400320 http.py:171] Sleeping 1.10 seconds before retry 1 of 5 for request: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json, after 502


Observation: 20/86
Observation: 30/86
Observation: 40/86
Observation: 50/86
Observation: 60/86
Observation: 70/86
Observation: 80/86
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/GM_3_1.tfrecord
Putting 72 observations into GM_4_1.tfrecord
Observation: 0/72
Observation: 10/72
Observation: 20/72
Observation: 30/72


In [ ]:
for row_i in range(0,10):
    print(row_i)
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki.iloc[[row_i]], SATELLITE, KERNEL_SIZE, year_i)

In [ ]:
survey_df_yeari_chunki.iloc[[1]]